In [1]:
# get data
from binance.spot import Spot 
from constants import price_cols, PERIOD, SYMBOL, DURATION_EACH_DAY, DATA_POINT_ONE_DAY, NUM_OF_DAYS

# train data
import pandas as pd
from datetime import datetime

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../../env/app.env")

False

In [2]:
#API key and connect
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)

## Read old prices

In [3]:
price = pd.read_csv(f"../../datastore/processed/{SYMBOL}_{PERIOD}.csv")
price # read from file

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1517760000000,8419.96,8532.83,8355.88,8386.77,472.122291,1517760899999,3.986791e+06,5060,215.227104,1.818350e+06
1,1517760900000,8386.77,8485.24,8300.00,8450.06,682.298603,1517761799999,5.702157e+06,6430,229.938351,1.923916e+06
2,1517761800000,8476.38,8495.01,8373.48,8467.62,424.080667,1517762699999,3.584546e+06,4384,235.532918,1.992314e+06
3,1517762700000,8441.29,8500.09,8370.02,8500.00,227.783558,1517763599999,1.917746e+06,3077,137.355614,1.156986e+06
4,1517763600000,8500.00,8579.00,8440.01,8543.98,322.068486,1517764499999,2.743974e+06,4304,189.538814,1.615451e+06
...,...,...,...,...,...,...,...,...,...,...,...
172333,1673327700000,17216.95,17223.34,17208.00,17215.23,1684.912670,1673328599999,2.900936e+07,41384,813.477680,1.400594e+07
172334,1673328600000,17215.23,17225.99,17207.05,17221.85,1363.170220,1673329499999,2.347062e+07,37643,663.414030,1.142270e+07
172335,1673329500000,17221.85,17228.64,17220.54,17227.44,200.039760,1673330399999,3.445478e+06,6599,107.728930,1.855552e+06
172336,1673330400000,17228.66,17233.50,17214.71,17220.39,1300.372320,1673331299999,2.239504e+07,34000,649.863570,1.119186e+07


## Fetch new prices

In [4]:
start_timestamp = price.iloc[-1, 0] # get the lastest time has in data
today = datetime.utcnow().replace(hour=23, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000) #the current time

In [5]:
# get new data
for timestamp in range(start_timestamp, today_timestamp, DURATION_EACH_DAY):
    data = client.klines(SYMBOL, PERIOD, limit=1000, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY)
    if start_timestamp == timestamp:
        new_price_df = pd.DataFrame(data, columns=price_cols)
    else:
        new_price_df = pd.concat([new_price_df, pd.DataFrame(data, columns=price_cols)], axis=0)

new_price_df.drop("Unused field, ignore", axis=1, inplace=True)

## Concatenate old and new prices and save

In [6]:
price = pd.concat([price, new_price_df])
price.drop_duplicates("Kline open time", inplace=True)
price.to_csv(f"../../datastore/processed/{SYMBOL}_{PERIOD}.csv", index=False)